In [8]:
import os
import NN
import utils
import config as cfg

### Setup Parameters

In [9]:
ETF = 'SPY' # 'SPY', 'DIA', 'QQQ'
NNtype = 'MLP' # 'MLP', 'RNN', 'PSN'
params = cfg.train_parameters[ETF][NNtype]
inputs_lag = cfg.SPYfeatures[NNtype] # SPYfeatures, DIAfeatures, QQQfeatures

### Load Dataframes

In [10]:
traindf = utils.load_file(os.path.join("data", ETF, NNtype, "Train.pkl"))
validdf = utils.load_file(os.path.join("data", ETF, NNtype, "Valid.pkl"))
testdf  = utils.load_file(os.path.join("data", ETF, NNtype, "Test.pkl"))

### Initialize DataLoaders

In [11]:
features_col = []
for i in inputs_lag:
    features_col.append("Return_" + str(i))
target_col   = 'Target'

mu, sigma = None, None

trainloader = utils.DataFrame2DataLoader(traindf, features_col, target_col, batch_size=10, normalize=False)
validloader = utils.DataFrame2DataLoader(validdf, features_col, target_col, batch_size=8, normalize=False)
testloader  = utils.DataFrame2DataLoader(testdf, features_col, target_col, batch_size=8, normalize=False)

### Instanciate Model

In [12]:
model = NN.Model(NNtype)
model.setup(params["input_size"], 
            params["hidden_size"],
            params["output_size"],
            1000,#params["epochs"],
            params["optim"],
            params["lr"],
            params["momentum"]
           )

Sequential(
  (0): Linear(in_features=7, out_features=6, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=6, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0.004
    nesterov: False
    weight_decay: 0
)


### Training

In [14]:
model.train(trainloader, validloader)

[00:00] Epoch 1/1000 - Train Loss : 0.0014 / Val Loss : 0.0005
[00:49] Epoch 1000/1000 - Train Loss : 0.0001 / Val Loss : 0.0000
tensor([[3.7056e-04],
        [3.3975e-04],
        [3.8487e-04],
        [4.5905e-04],
        [3.4350e-04],
        [4.5180e-05],
        [4.8345e-04],
        [4.8125e-04],
        [4.3628e-04],
        [3.4302e-04]])
tensor([[ 4.0692e-04],
        [ 3.1942e-04],
        [-2.0862e-06],
        [ 5.3513e-04],
        [ 3.7920e-04],
        [ 2.9761e-04],
        [ 6.5142e-04],
        [ 4.1631e-04],
        [ 1.5813e-04],
        [ 9.0241e-05]])
tensor([[ 6.8724e-04],
        [ 6.5738e-04],
        [ 1.6993e-04],
        [ 7.9912e-04],
        [ 2.7460e-04],
        [-1.7643e-04],
        [ 7.5379e-04],
        [ 5.9488e-04],
        [-4.8995e-05],
        [ 1.7309e-04]])
tensor([[5.2404e-04],
        [3.3176e-04],
        [2.6101e-04],
        [1.9503e-04],
        [3.0255e-04],
        [3.1817e-04],
        [7.3910e-05],
        [3.4523e-04],
        [6.5

/Users/Chayan/Dev/projects/PortfolioOptimization/utils.py:17: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred)/y_true))
/Users/Chayan/Dev/projects/PortfolioOptimization/utils.py:17: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred)/y_true))


### Evaluation

In [15]:
model.evaluate(testloader)

Test MAE : 0.0055 | Test MAPE  : 1.0801 | Test RSME : 0.0001 | Test Theil-U 0.9353
